In [2]:
import sys, os

# if your notebook lives in LAB3/notebooks and src/ is in LAB3/src,
# this will add the LAB3 folder into Python’s search path:
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [3]:
import pandas as pd
import random
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error
import tqdm as notebook_tqdm
from src.metrics import print_error_rate_total_y_cat

In [5]:
# Cargar el dataset
df = pd.read_csv('../data/ventas_featurev4.csv', sep=',')

In [6]:
SEED = 26461180
random.seed(SEED)
np.random.seed(SEED)

# Convertir 'periodo' de fecha a entero tipo YYYYMM
df["periodo"] = pd.to_datetime(df["periodo"])
df["periodo"] = df["periodo"].dt.strftime("%Y%m").astype(int)

# Convierte todas las columnas object a category
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')
    
df = df.sort_values(['product_id','periodo'])

# Convertir todas las columnas float64 a float16 en sell_in_featured1
float64_cols = df.select_dtypes(include=['float64']).columns
df[float64_cols] = df[float64_cols].astype('float16')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16763760 entries, 0 to 16763759
Data columns (total 97 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   periodo                    int32   
 1   customer_id                int64   
 2   product_id                 int64   
 3   plan_precios_cuidados      float16 
 4   tn                         float16 
 5   cust_request_qty           float16 
 6   cust_request_tn            float16 
 7   cat1                       category
 8   cat2                       category
 9   cat3                       category
 10  brand                      category
 11  sku_size                   int64   
 12  descripcion                category
 13  productos_estrella         int64   
 14  mes                        int64   
 15  catastrofe                 int64   
 16  cliente_estrella           int64   
 17  prod_cust                  category
 18  tn_lag_1                   float16 
 19  tn_lag_2           

In [7]:
# 3) Define target and features
label_col = 'tn_mas_2'
y = df[label_col].astype('float32')
X = df.drop(columns=[label_col])
X = X.drop(columns=['descripcion','prod_cust'])


In [8]:
from sklearn.model_selection import TimeSeriesSplit

test_mask = df['periodo'] == 201912
X_tv, y_tv = X[~test_mask], y[~test_mask]
X_test, y_test = X[test_mask], y[test_mask]

# time-series CV on train+val
tscv = TimeSeriesSplit(
    n_splits=3,
    max_train_size=None,   # grows the training window
    test_size=1            # one period per fold
)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_tv)):
    print(f"Fold {fold} → train: {df.periodo.iloc[train_idx].min()}–"
          f"{df.periodo.iloc[train_idx].max()},  val: "
          f"{df.periodo.iloc[val_idx].unique()}")
    X_train, X_val = X_tv.iloc[train_idx], X_tv.iloc[val_idx]
    y_train, y_val = y_tv.iloc[train_idx], y_tv.iloc[val_idx]
    


Fold 0 → train: 201701–201912,  val: [201712]
Fold 1 → train: 201701–201912,  val: [201712]
Fold 2 → train: 201701–201912,  val: [201712]


In [9]:
def objective(trial):
    # fixed estimators; early stopping handled in .fit()
    params = {
        'n_estimators': 5_000,
        'objective':   'regression',
        'random_state': SEED,
        'boosting_type': 'gbdt',
        'num_leaves':    trial.suggest_int('num_leaves', 31, 512),
        'max_depth':     trial.suggest_int('max_depth', 6, 30),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'feature_fraction':  trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction':  trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq':      trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1':         trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2':         trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[
            lgb.early_stopping(stopping_rounds=10)
        ]
    )
    preds = model.predict(X_val)
    mse  = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    return rmse

In [10]:
study = optuna.create_study(direction='minimize', study_name='lgbm_tn_mas_2')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best RMSE on 201911:", study.best_value)
print("Best params:", study.best_params)

# 7) Retrain on train+val and forecast Feb-2020
best_params = study.best_params.copy()
best_params.update({
    'n_estimators': study.best_trial.user_attrs.get('n_estimators', 5_000),
    'objective': 'regression',
    'random_state': SEED
})

[I 2025-06-30 15:08:41,012] A new study created in memory with name: lgbm_tn_mas_2
  0%|          | 0/50 [00:00<?, ?it/s]

[LightGBM] [Warning] feature_fraction is set=0.6294239575420411, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6294239575420411
[LightGBM] [Warning] lambda_l1 is set=1.3737457477688024e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3737457477688024e-07
[LightGBM] [Warning] lambda_l2 is set=6.586502379022086e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.586502379022086e-06
[LightGBM] [Warning] bagging_fraction is set=0.6385652770041821, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6385652770041821
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.6294239575420411, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6294239575420411
[LightGBM] [Warning] lambda_l1 is set=1.3737457477688024e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3737457477688024e-07
[LightGBM] [War

Best trial: 0. Best value: 0.0672866:   2%|▏         | 1/50 [00:48<39:15, 48.07s/it]

[LightGBM] [Warning] feature_fraction is set=0.6294239575420411, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6294239575420411
[LightGBM] [Warning] lambda_l1 is set=1.3737457477688024e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3737457477688024e-07
[LightGBM] [Warning] lambda_l2 is set=6.586502379022086e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.586502379022086e-06
[LightGBM] [Warning] bagging_fraction is set=0.6385652770041821, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6385652770041821
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[I 2025-06-30 15:09:29,115] Trial 0 finished with value: 0.06728661418666396 and parameters: {'num_leaves': 221, 'max_depth': 10, 'learning_rate': 0.0014618940792473023, 'min_child_samples': 179, 'feature_fraction': 0.6294239575420411, 'bagging_fraction': 0.6385652770041821, 'bagging_freq': 4, 'lambda_l1': 1.

Best trial: 1. Best value: 0.0234132:   4%|▍         | 2/50 [03:17<1:26:11, 107.74s/it]

...

[LightGBM] [Warning] feature_fraction is set=0.9448824778233517, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9448824778233517
[LightGBM] [Warning] lambda_l1 is set=0.010470076140057112, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.010470076140057112
[LightGBM] [Warning] lambda_l2 is set=1.2338181163208567e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2338181163208567e-07
[LightGBM] [Warning] bagging_fraction is set=0.5551509448500324, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5551509448500324
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[I 2025-06-30 16:08:47,207] Trial 47 finished with value: 0.02439279227929765 and parameters: {'num_leaves': 78, 'max_depth': 13, 'learning_rate': 0.056163380619090965, 'min_child_samples': 42, 'feature_fraction': 0.9448824778233517, 'bagging_fraction': 0.5551509448500324, 'bagging_freq': 3, 'lambda_l1': 0.0104

Best trial: 42. Best value: 0.0156358:  98%|█████████▊| 49/50 [1:00:44<01:16, 76.75s/it]

[LightGBM] [Warning] feature_fraction is set=0.9725933602233975, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9725933602233975
[LightGBM] [Warning] lambda_l1 is set=0.00046180770976404556, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00046180770976404556
[LightGBM] [Warning] lambda_l2 is set=2.2756705551919818e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.2756705551919818e-08
[LightGBM] [Warning] bagging_fraction is set=0.5249494884666552, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5249494884666552
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[I 2025-06-30 16:09:25,788] Trial 48 finished with value: 0.045093767005527696 and parameters: {'num_leaves': 312, 'max_depth': 24, 'learning_rate': 0.08170858654866929, 'min_child_samples': 70, 'feature_fraction': 0.9725933602233975, 'bagging_fraction': 0.5249494884666552, 'bagging_freq': 9, 'lambda_l1': 0

Best trial: 42. Best value: 0.0156358: 100%|██████████| 50/50 [1:01:20<00:00, 73.62s/it]

[LightGBM] [Warning] feature_fraction is set=0.6947200909687608, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6947200909687608
[LightGBM] [Warning] lambda_l1 is set=0.00021767892392113552, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00021767892392113552
[LightGBM] [Warning] lambda_l2 is set=3.9414107832500596e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.9414107832500596e-07
[LightGBM] [Warning] bagging_fraction is set=0.7106651171108893, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7106651171108893
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[I 2025-06-30 16:10:01,878] Trial 49 finished with value: 0.06471947459803751 and parameters: {'num_leaves': 424, 'max_depth': 11, 'learning_rate': 0.04323964550881397, 'min_child_samples': 27, 'feature_fraction': 0.6947200909687608, 'bagging_fraction': 0.7106651171108893, 'bagging_freq': 7, 'lambda_l1': 0.

In [12]:
final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(
    pd.concat([X_train, X_val]),
    pd.concat([y_train, y_val]),
    eval_metric='rmse'
)

preds_202002 = final_model.predict(X_test)
results = df.loc[test_mask, ['product_id','periodo']].copy()
results['pred_tn_202002'] = preds_202002

print(results.head())

# results.to_csv('../data/lgbm_forecast_202002_per_product.csv', index=False)

[LightGBM] [Warning] feature_fraction is set=0.956171264589964, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.956171264589964
[LightGBM] [Warning] lambda_l1 is set=0.00022935117286783794, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00022935117286783794
[LightGBM] [Warning] lambda_l2 is set=4.7040833615196834e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.7040833615196834e-08
[LightGBM] [Warning] bagging_fraction is set=0.5421463876423206, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5421463876423206
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.956171264589964, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.956171264589964
[LightGBM] [Warning] lambda_l1 is set=0.00022935117286783794, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00022935117286783794
[LightGBM] [Warni

In [ ]:
results['forecast_period'] = (
    pd.to_datetime(results['periodo'].astype(str), format='%Y%m')
      .add(pd.DateOffset(months=2))
      .dt.strftime('%Y%m')
      .astype(int)
)

prod_forecast = (
    results
      .groupby(['product_id','forecast_period'], as_index=False)
      ['pred_tn_202002']
      .sum()
      .rename(columns={'pred_tn_202002':'agg_pred_tn_202002'})
)

max_period = df['periodo'].max()  # should be 201912
# get the three prior periods dynamically
ts_max = pd.to_datetime(str(max_period), format='%Y%m')
last3 = [
    (ts_max - pd.DateOffset(months=i)).strftime('%Y%m')
    for i in (1,2,3)
]
last3 = [int(x) for x in last3]  # [201911,201910,201909]

hist_mean = (
    df[df['periodo'].isin(last3)]
      .groupby('product_id')['tn']
      .mean()
      .reset_index()
      .rename(columns={'tn':'fallback_mean_last3'})
)

prod_forecast = prod_forecast.merge(hist_mean, on='product_id', how='left')
prod_forecast['final_forecast'] = prod_forecast.apply(
    lambda r: r['fallback_mean_last3']
              if r['agg_pred_tn_202002'] < 0
              else r['agg_pred_tn_202002'],
    axis=1
)

final = prod_forecast[[
    'product_id',
    'forecast_period',
    'final_forecast'
]].rename(columns={'final_forecast':'pred_tn_202002'})

output = (
    final
      .loc[:, ['product_id', 'pred_tn_202002']]
      .rename(columns={'pred_tn_202002': 'tn'})
)



print(output.head())

# 7) Persist to CSV
output.to_csv('../data/lgbm_forecast_202002_per_productv4.csv', index=False)

   product_id           tn
0       20001  1444.044956
1       20002  1097.350068
2       20003  1053.255103
3       20004   767.045802
4       20005   598.838563
